In [1]:
import pandas

In [2]:
#model Parameters
growth_factor= 0.18
EBITDA_margin = 0.45
tax_pct = 0.21
cap_ex = 0.03
work_cap_growth = 0.06
discount_factor = 0.94
exit_multiple = 32.70

In [3]:
dates = pandas.date_range(start='2022', end = '2032' , freq='Y')
dates

DatetimeIndex(['2022-12-31', '2023-12-31', '2024-12-31', '2025-12-31',
               '2026-12-31', '2027-12-31', '2028-12-31', '2029-12-31',
               '2030-12-31', '2031-12-31'],
              dtype='datetime64[ns]', freq='A-DEC')

In [4]:
model_df=pandas.DataFrame(index=dates)
terminal_value_df=pandas.DataFrame()
model_df

""
2022-12-31
2023-12-31
2024-12-31
2025-12-31
2026-12-31
2027-12-31
2028-12-31
2029-12-31
2030-12-31
2031-12-31


In [5]:
#Rev Growth

start_rev = 222

rev_projection = [start_rev * growth_factor]
for i in range (1,len(dates)):
    rev_projection.append(rev_projection[i-1]*growth_factor)
model_df['revenue']  = rev_projection

In [6]:
model_df['EBITDA'] = model_df['revenue']*EBITDA_margin

In [7]:
model_df['taxes'] = model_df['EBITDA']* - tax_pct

In [8]:
model_df['cap_ex'] = model_df['revenue']* - cap_ex

In [9]:
model_df['work_cap'] = model_df['revenue']* - work_cap_growth


In [10]:
model_df['cash_flow'] = model_df.drop(columns=['revenue']).sum(axis=1)

In [11]:
discount_factors=[discount_factor**i for i in range (1,len(dates)+1)]
discount_factors*model_df['cash_flow'].values

array([9.97281720e+00, 1.68740067e+00, 2.85508193e-01, 4.83079863e-02,
       8.17371129e-03, 1.38299195e-03, 2.34002238e-04, 3.95931786e-05,
       6.69916583e-06, 1.13349886e-06])

In [12]:
discount_factors=[discount_factor**i for i in range (1,len(dates)+1)]
discount_table=pandas.DataFrame(index=discount_factors)
discount_table['present_value_fcf'] = discount_factors*model_df['cash_flow'].values 

In [13]:
present_value_of_projected_fcf = discount_table['present_value_fcf']. sum()
present_value_of_projected_fcf

12.003872181285496

In [14]:
terminal_frame = pandas.DataFrame(index=['2032'])
terminal_frame['revenue'] = model_df['revenue'].iloc[-1]*growth_factor
terminal_frame['EBITDA' ] = terminal_frame['revenue']*EBITDA_margin 


In [15]:
terminal_frame['terminal_valuation']=terminal_frame['EBITDA']*exit_multiple
terminal_frame['terminal_valuation'].round(0)

2032    0.0
Name: terminal_valuation, dtype: float64

In [17]:
terminal_frame['revenue']



2032    0.000001
Name: revenue, dtype: float64

In [18]:
terminal_frame['EBITDA'] 

2032    6.420414e-07
Name: EBITDA, dtype: float64

"Our projected Present Value of Projected Future Cash Flows are:12.


We are providing revenue guidance in 2032 to be:  0.000001.


With an estimated EBITDA around:6.420414e-07.
